In [1]:
import pandas as pd
import numpy as np
import nltk

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import pyLDAvis
import pyLDAvis.gensim_models

In [4]:
#1.Read the .csv file using Pandas. Take a look at the top few records.
data=pd.read_csv(r"C:\Users\hp\Downloads\NLP\My_Projects\1.Topic Analysis of Review Data\K8 Reviews v0.2.csv")
data.head()

C:\Users\hp\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,sentiment,review
0,1,Good but need updates and improvements
1,0,"Worst mobile i have bought ever, Battery is dr..."
2,1,when I will get my 10% cash back.... its alrea...
3,1,Good
4,0,The worst phone everThey have changed the last...


In [5]:
data['sentiment'].value_counts()

C:\Users\hp\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0    7712
1    6963
Name: sentiment, dtype: int64

In [6]:
#2.Normalize casings for the review text and extract the text into a list for easier manipulation
review=data['review'].values
review[:5]

C:\Users\hp\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


array(['Good but need updates and improvements',
       "Worst mobile i have bought ever, Battery is draining like hell, backup is only 6 to 7 hours with internet uses, even if I put mobile idle its getting discharged.This is biggest lie from Amazon & Lenove which is not at all expected, they are making full by saying that battery is 4000MAH & booster charger is fake, it takes at least 4 to 5 hours to be fully charged.Don't know how Lenovo will survive by making full of us.Please don;t go for this else you will regret like me.",
       'when I will get my 10% cash back.... its already 15 January..',
       'Good',
       'The worst phone everThey have changed the last phone but the problem is still same and the amazon is not returning the phone .Highly disappointing of amazon'],
      dtype=object)

In [7]:
review0=[i.lower() for i in review]
review0[:5]

C:\Users\hp\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


['good but need updates and improvements',
 "worst mobile i have bought ever, battery is draining like hell, backup is only 6 to 7 hours with internet uses, even if i put mobile idle its getting discharged.this is biggest lie from amazon & lenove which is not at all expected, they are making full by saying that battery is 4000mah & booster charger is fake, it takes at least 4 to 5 hours to be fully charged.don't know how lenovo will survive by making full of us.please don;t go for this else you will regret like me.",
 'when i will get my 10% cash back.... its already 15 january..',
 'good',
 'the worst phone everthey have changed the last phone but the problem is still same and the amazon is not returning the phone .highly disappointing of amazon']

In [8]:
#3.Tokenize the reviews using NLTKs word_tokenize function.
from nltk.tokenize import word_tokenize

C:\Users\hp\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [9]:
print(review0[0])

good but need updates and improvements


C:\Users\hp\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [10]:
rev_tokens=[word_tokenize(i) for i in review0]
print(rev_tokens[0])

C:\Users\hp\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


['good', 'but', 'need', 'updates', 'and', 'improvements']


In [11]:
#4.Perform parts-of-speech tagging on each sentence using the NLTK POS tagger.

from nltk import pos_tag
rev_pos=[pos_tag(i) for i in rev_tokens]


C:\Users\hp\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [12]:
rev_pos[2]

C:\Users\hp\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[('when', 'WRB'),
 ('i', 'NN'),
 ('will', 'MD'),
 ('get', 'VB'),
 ('my', 'PRP$'),
 ('10', 'CD'),
 ('%', 'NN'),
 ('cash', 'NN'),
 ('back', 'RB'),
 ('....', 'VBZ'),
 ('its', 'PRP$'),
 ('already', 'RB'),
 ('15', 'CD'),
 ('january', 'JJ'),
 ('..', 'NN')]

In [13]:
#5.For the topic model, we should  want to include only nouns.
    #Find out all the POS tags that correspond to nouns.
    #Limit the data to only terms with these tags.

nouns=[]
for row in rev_pos:
    for token,pos in row:
        if pos.startswith('N'):
            nouns.append(token)
nouns[:10]

    

C:\Users\hp\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


['updates',
 'improvements',
 'mobile',
 'i',
 'battery',
 'hell',
 'backup',
 'hours',
 'uses',
 'idle']

In [14]:
#6.Lemmatize
from nltk.stem import WordNetLemmatizer
lemmatize=WordNetLemmatizer()
rev_lem=[lemmatize.lemmatize(words) for words in nouns]

C:\Users\hp\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [15]:
rev_lem[:10]

C:\Users\hp\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


['update',
 'improvement',
 'mobile',
 'i',
 'battery',
 'hell',
 'backup',
 'hour',
 'us',
 'idle']

In [16]:
#7.Remove stopwords and punctuation 
from nltk.corpus import stopwords
from string import punctuation

C:\Users\hp\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [17]:
stop_nltk=stopwords.words('english')
stop_punc=list(punctuation)
stop_punc.extend(['...','``',"''",".."])

C:\Users\hp\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [18]:
stop_final=stop_nltk+stop_punc
def del_stop(sent):
    return [term for term in sent if term not in stop_final]

C:\Users\hp\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [19]:
del_stop(rev_lem[:10])

C:\Users\hp\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


['update',
 'improvement',
 'mobile',
 'battery',
 'hell',
 'backup',
 'hour',
 'us',
 'idle']

In [20]:
rev_final=del_stop(rev_lem)

C:\Users\hp\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [21]:
rev_final[:10]

C:\Users\hp\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


['update',
 'improvement',
 'mobile',
 'battery',
 'hell',
 'backup',
 'hour',
 'us',
 'idle',
 'discharged.this']

In [22]:
len(rev_final)

C:\Users\hp\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


87740

# Use counter to get top nouns/keywords for all the reviews

In [23]:
from collections import Counter

C:\Users\hp\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [24]:
all_clean= Counter(rev_final)
all_clean.most_common(15)

C:\Users\hp\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[('phone', 7007),
 ('camera', 3273),
 ('battery', 3143),
 ('product', 2261),
 ('problem', 1565),
 ('mobile', 1517),
 ('issue', 1490),
 ('quality', 1387),
 ('note', 1163),
 ('lenovo', 1003),
 ('time', 1003),
 ('performance', 952),
 ('price', 924),
 ('day', 897),
 ('feature', 841)]

In [25]:
#8.Create a topic model using LDA on the cleaned-up data with 12 topics
import gensim
from gensim import corpora
rev_out=[rev_final]
dictionary=corpora.Dictionary(rev_out)

C:\Users\hp\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
C:\Users\hp\anaconda3\lib\site-packages\requests\packages\urllib3\util\selectors.py:11: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.9 it will stop working
  from collections import namedtuple, Mapping
C:\Users\hp\anaconda3\lib\site-packages\requests\packages\urllib3\_collections.py:2: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.9 it will stop working
  from collections import Mapping, MutableMapping
C:\Users\hp\anaconda3\lib\site-packages\scipy\

In [26]:
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from pprint import pprint

C:\Users\hp\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [27]:
import pyLDAvis
import pyLDAvis.sklearn
import matplotlib.pyplot as plt
%matplotlib inline

C:\Users\hp\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
C:\Users\hp\anaconda3\lib\site-packages\matplotlib\__init__.py:169: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(module.__version__) < minver:
C:\Users\hp\anaconda3\lib\site-packages\setuptools\_distutils\version.py:346: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)
C:\Users\hp\anaconda3\lib\site-packages\matplotlib\__init__.py:169: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(module.__version__) < minver:
C:\Users\hp\anaconda3\lib\

In [28]:
vectorizer = CountVectorizer(analyzer='word',       
                             min_df=10,                        # minimum reqd occurences of a word 
                             stop_words='english',             # remove stop words
                                               # convert all words to lowercase
                             token_pattern='[a-zA-Z0-9]{3,}',  # num chars > 3
                             # max_features=50000,             # max number of uniq words
                            )


C:\Users\hp\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [29]:
rev_new = [word.lower() for word in rev_final]
data_vectorized = vectorizer.fit_transform(rev_final)

C:\Users\hp\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [35]:
lda_model=LatentDirichletAllocation(n_components=12)

lda_output = lda_model.fit_transform(data_vectorized)
print(lda_model)

C:\Users\hp\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


LatentDirichletAllocation(n_components=12)


In [36]:
# See model parameters
pprint(lda_model.get_params())

C:\Users\hp\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Perplexity:  186.02649177216912
{'batch_size': 128,
 'doc_topic_prior': None,
 'evaluate_every': -1,
 'learning_decay': 0.7,
 'learning_method': 'batch',
 'learning_offset': 10.0,
 'max_doc_update_iter': 100,
 'max_iter': 10,
 'mean_change_tol': 0.001,
 'n_components': 12,
 'n_jobs': None,
 'perp_tol': 0.1,
 'random_state': None,
 'topic_word_prior': None,
 'total_samples': 1000000.0,
 'verbose': 0}


In [37]:
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(lda_model, data_vectorized, vectorizer, mds='tsne')
panel

C:\Users\hp\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
5        5.310529  171.049652       1        1  10.666850
2      136.304657   23.800941       2        1   9.511105
4      101.190552  -84.985191       3        1   9.340126
9     -106.824867  148.841736       4        1   8.157095
7      -44.365761   78.632721       5        1   8.151005
8       43.884090   73.474022       6        1   8.093153
3      114.093971  135.933029       7        1   7.883295
6     -119.727142  -72.077179       8        1   7.804598
10     -49.518208   -9.617325       9        1   7.772510
1     -141.938736   40.055210      10        1   7.649546
11      38.731544  -14.776019      11        1   7.600829
0      -10.944340 -107.193344      12        1   7.369890, topic_info=        Term         Freq        Total Category  logprob  loglift
436    phone  5889.000000  5889.000000  Default  30.0000  30.0000
42   battery  3307.000000  3307.000000  Default  29.0000  29.0000
76    camera  3045.000000  3045.000000  Default  28.0000  28.0000
473  product  2355.000000  2355.000000  Default  27.0000  27.0000
467  problem  1688.000000  1688.000000  Default  26.0000  26.0000
..       ...          ...          ...      ...      ...      ...
452   pocket    40.907933    41.824051  Topic12  -4.8536   2.5856
178     edge    38.640518    39.556637  Topic12  -4.9106   2.5843
144  default    36.373104    37.289223  Topic12  -4.9711   2.5829
95     check    32.971983    33.888101  Topic12  -5.0692   2.5804
567     song    30.704568    31.620687  Topic12  -5.1405   2.5784

[390 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
3         6  0.978439    accessory
15        3  0.998742       amazon
17        5  0.997374      android
19        6  0.995511          app
20        3  0.990437  application
...     ...       ...          ...
668      10  0.971561      working
671       1  0.993138        worth
677      12  0.988347         year
679       2  0.984133    yesterday
680       6  0.978265      youtube

[360 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 3, 5, 10, 8, 9, 4, 7, 11, 2, 12, 1])